<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/finding_magic_feature_id.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "tapaskd123" # username from the json file
os.environ['KAGGLE_KEY'] = "aba8dc1f085221111d925003fe5a88ed" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 52.5MB/s]
 82% 43.0M/52.2M [00:00<00:00, 64.8MB/s]
100% 52.2M/52.2M [00:00<00:00, 117MB/s] 
 89% 52.0M/58.3M [00:00<00:00, 57.0MB/s]
100% 58.3M/58.3M [00:00<00:00, 108MB/s] 
  0% 0.00/3.26M [00:00<?, ?B/s]
100% 3.26M/3.26M [00:00<00:00, 107MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 164MB/s]


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Mounted at /content/gdrive


In [3]:
import pandas as pd
import numpy as np
trn=pd.read_csv('train_transaction.csv.zip')
trn_idn=pd.read_csv('train_identity.csv.zip')

In [4]:
trn=trn.merge(trn_idn,on='TransactionID',how='left')
import gc
ls=list(trn_idn)
del([trn_idn])
gc.collect()

11

In [5]:
trn['day']=trn['TransactionDT']//86400
for col in range(1,16):
  trn['d_'+str(col)]=trn['day']-trn['D'+str(col)]

In [6]:
trn.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,day,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,-13.0,NaN,-12.0,NaN,NaN,NaN,NaN,NaN,NaN,-12.0,-12.0,NaN,NaN,NaN,1.0
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,-314.0,NaN,NaN,NaN,-314.0
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,-111.0,-111.0,1.0,-93.0,1.0,NaN,NaN,NaN,NaN,-83.0,NaN,NaN,NaN,NaN,-110.0
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
ls.append('TransactionAmt')
ls.remove('TransactionID')
ls.append('V307')
ls

['id_01',
 'id_02',
 'id_03',
 'id_04',
 'id_05',
 'id_06',
 'id_07',
 'id_08',
 'id_09',
 'id_10',
 'id_11',
 'id_12',
 'id_13',
 'id_14',
 'id_15',
 'id_16',
 'id_17',
 'id_18',
 'id_19',
 'id_20',
 'id_21',
 'id_22',
 'id_23',
 'id_24',
 'id_25',
 'id_26',
 'id_27',
 'id_28',
 'id_29',
 'id_30',
 'id_31',
 'id_32',
 'id_33',
 'id_34',
 'id_35',
 'id_36',
 'id_37',
 'id_38',
 'DeviceType',
 'DeviceInfo',
 'TransactionAmt',
 'V307']

In [8]:
trn=trn.drop(['TransactionID','TransactionDT','isFraud'],1)
cat=list(trn.select_dtypes(include=object))
trn[cat]=trn[cat].fillna('nan')
trn=trn.fillna(-999)

In [9]:
trn=trn.loc[trn['D1']<trn['day']].reset_index(drop=True)

In [10]:
from tqdm import tqdm
import random
def group(df,cols):
  df['id']=''
  for col in cols:
    df['id']+=df[col].astype(str)
  result=0
  for _,sdo in df.groupby(['id']):
    tmp=0
    tmp+=sdo['TransactionAmt'].values.sum()
    tmp-=sdo.tail(1)['V307'].values
    if tmp>0:
      result+=tmp
    else:
      tmp-=result
  return result
trn=trn[ls]
cols=list(trn)
cols.remove('TransactionAmt')
cols.remove('V307')
random.shuffle(cols)
ans=0
ls=[]
dk={}
for en,col in enumerate(tqdm(cols)):
  ls.append(col)
  n_ans=group(trn,ls)
  dk[col]=ans-n_ans
  if ans!=0:
    if n_ans>=ans:
      ls.remove(col)
    else:
      ans=n_ans
  else:
    ans=n_ans

100%|██████████| 40/40 [29:37<00:00, 44.44s/it]


In [11]:
ans

array([55890610.57003769])

In [12]:
df=pd.DataFrame(dk).T
df['rank']=list(range(len(cols)))
df.sort_values(by=0,ascending=False).head(20)

,0,rank
id_02,962139.645486,16
id_20,604547.952839,9
id_31,411388.261419,7
id_19,345282.036014,6
id_12,227777.592498,14
id_08,70790.972801,5
id_14,47140.360309,13
id_33,22315.831970,1
id_11,18688.498007,2
id_03,15142.821897,3


In [13]:
ls

['id_29',
 'id_33',
 'id_11',
 'id_03',
 'id_24',
 'id_08',
 'id_19',
 'id_31',
 'id_25',
 'id_20',
 'id_28',
 'id_16',
 'id_14',
 'id_12',
 'id_02',
 'id_05',
 'id_13',
 'id_01',
 'id_34']